# Bootstrap de ações com IPO em 2021

# 1) Storyteling

O mercado sempre teve a impressão de preço esticado no IPO de empresas.

Publicado pelo estatístico Bradley Efron em 1979 no artigo "Bootstrap methods: another look at the jackknife", o Bootstrap é um método de reamostragem com reposição mais "computacional" que o seu predecessor, o jackknife.

Nesse bootstrap, utilizarei o preço das 45 empresas com capital aberto em 2021. Calcularei médias, medianas e desvios para estimar e comparar a variância dos preços.

Na conclusão, observamos que somente 11 empresas (com destaque aos setores de agronegócio e tecnologia) estão atualmente cotadas a preços maiores que o seu IPO.

Ademais, as máximas históricas apontam para estratégia de trading short na queda dos preços pós IPO e na queda drástica dos preços pós máxima histórica.  




Link das 45 empresas com IPO em 2021

https://www.poder360.com.br/economia/75-das-empresas-que-fizeram-ipos-em-2021-tem-acoes-no-vermelho/#:~:text=Em%202021%2C%20a%20B3%20


# 2) Bibliotecas

In [2]:
!pip install yfinance -q
import yfinance as yf
import pandas as pd
import numpy as np
from sklearn.utils import resample

# 3) Desenvolvendo o algoritmo

In [3]:
tickers = ['WEST3.SA', 'ESPA3.SA', 'DOTZ3.SA', 'MBLY3.SA', 'NINJ3.SA', 'CLSA3.SA', 'BRIT3.SA', 'LVTC3.SA', 'TRAD3.SA', 'OPCT3.SA', 'KRSA3.SA', 'HBRE3.SA',
           'CSED3.SA', 'IFCM3.SA', 'ONCO3.SA', 'MLAS3.SA', 'FIQE3.SA', 'ALLD3.SA', 'NUBR33.SA', 'DESK3.SA', 'CMIN3.SA', 'G2DI33.SA', 'MATD3.SA', 'RAIZ4.SA',
           'AGXY3.SA','ELMD3.SA', 'BMOB3.SA', 'SMFT3.SA', 'BLAU3.SA', 'VVEO3.SA', 'ARML3.SA', 'CXSE3.SA', 'JALL3.SA', 'BRBI11.SA', 'TTEN3.SA', 'IGTI3.SA',
           'CBAV3.SA', 'GGPS3.SA', 'SOJA3.SA', 'VITT3.SA', 'ORVR3.SA', 'INTB3.SA', 'RECV3.SA', 'VAMO3.SA', 'ASAI3.SA']

tickers.sort()
len(tickers)


45

In [4]:
# Baixar cotações de empresas com IPO a partir de 2021

cotacoes_ipo_2021 = yf.download(tickers, start='2021-01-01')

[*********************100%***********************]  45 of 45 completed


In [5]:
# Dos dados OHLC, somente o adjusted close é necessário para esse estudo
cotacoes_ipo_2021_ac = round(cotacoes_ipo_2021['Adj Close'], 2)
cotacoes_ipo_2021_ac.tail()

,AGXY3.SA,ALLD3.SA,ARML3.SA,ASAI3.SA,BLAU3.SA,BMOB3.SA,BRBI11.SA,BRIT3.SA,CBAV3.SA,CLSA3.SA,...,RAIZ4.SA,RECV3.SA,SMFT3.SA,SOJA3.SA,TRAD3.SA,TTEN3.SA,VAMO3.SA,VITT3.SA,VVEO3.SA,WEST3.SA
Date,,,,,,,,,,,,,,,,,,,,,
2023-07-28,5.41,6.84,12.86,13.20,20.45,13.50,14.70,2.93,4.98,5.54,...,4.12,23.16,22.75,11.38,1.50,12.28,11.71,12.30,21.11,1.38
2023-07-31,5.45,6.85,13.12,13.47,20.78,13.50,14.61,3.02,5.30,5.57,...,4.16,22.99,22.86,11.62,1.50,12.65,11.94,12.32,21.20,1.38
2023-08-01,5.35,6.77,13.45,13.65,20.93,13.50,15.50,3.05,5.29,5.77,...,4.16,22.92,23.10,11.82,1.48,12.22,12.01,12.32,21.67,1.26
2023-08-02,5.22,6.83,13.07,13.65,21.51,13.79,16.00,3.01,5.12,5.40,...,4.19,22.60,23.47,11.70,1.44,12.51,11.77,12.30,21.00,1.37
2023-08-03,5.09,6.90,13.27,14.11,21.44,13.93,16.33,3.08,5.17,5.38,...,4.13,23.08,23.29,11.69,1.47,12.50,12.08,12.10,20.83,1.40


In [6]:
# Não é necessário tratamento especial a missing values pois, simplesmente, quanto mais nan numa ação, mais recente o seu IPO.

cotacoes_ipo_2021_ac.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 645 entries, 2021-01-04 to 2023-08-03
Data columns (total 45 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   AGXY3.SA   506 non-null    float64
 1   ALLD3.SA   579 non-null    float64
 2   ARML3.SA   504 non-null    float64
 3   ASAI3.SA   608 non-null    float64
 4   BLAU3.SA   574 non-null    float64
 5   BMOB3.SA   618 non-null    float64
 6   BRBI11.SA  514 non-null    float64
 7   BRIT3.SA   502 non-null    float64
 8   CBAV3.SA   513 non-null    float64
 9   CLSA3.SA   502 non-null    float64
 10  CMIN3.SA   613 non-null    float64
 11  CSED3.SA   618 non-null    float64
 12  CXSE3.SA   566 non-null    float64
 13  DESK3.SA   509 non-null    float64
 14  DOTZ3.SA   545 non-null    float64
 15  ELMD3.SA   616 non-null    float64
 16  ESPA3.SA   623 non-null    float64
 17  FIQE3.SA   504 non-null    float64
 18  G2DI33.SA  554 non-null    float64
 19  GGPS3.SA   568 non-null    floa

In [7]:
# Criar o bootstrap

for coluna_ticker, cotac_hist in cotacoes_ipo_2021_ac.items():                                                # Iterar sobre a coluna ticker de cada empresa, em suas cotações históricas
  results = []

  for nrepeat in range(1000):                                                                                 # Iterar mil repetições com reposição pelo método resample do sklearn
    amostra = resample(cotac_hist)
    results.append(amostra.median())
  results = pd.Series(results)
  print(coluna_ticker,':', round(cotac_hist[-1],2))                                                           # Ticker: cotação atual
  print('Cotação 1º dia de IPO: ', round(cotac_hist.loc[~cotac_hist.isnull()][0],2))
  print('Mínima histórica: ', round(cotac_hist.min(),2),'. Máxima histórica:', round(cotac_hist.max(),2))     # Cotações mínima e máxima
  print('Variação Máxima % desde o IPO: ', round((cotac_hist.max() - cotac_hist[-1]) / cotac_hist[-1]*100,2))
  print('Média da cotação histórica indica o seguinte preço: ', round(cotac_hist.mean(),2))                   # Média da cotação histórica
  print('Mediana da cotação histórica indica o seguinte preço: ', round(cotac_hist.median(),2))               # Mediana da cotação histórica
  print('Mediana do Bootstrap indica o seguinte preço: ', round(results.median(),2))
  print('Distribuição do Bootstrap comparado à cotação histórica estima um viés de: ', round(results.mean() - cotac_hist.median(),2), 'no preço atual')
  print('Desvio Padrão do Bootstrap: ', round(results.std(),2), '. Isso indica um preço histórico entre:', round(results.mean() - results.std(),2), ' e:', round(results.mean() + results.std(),2))
  print('Isso indica uma variação percentual média, desde o IPO', 'de: ', (results.mean() + results.std() - (results.mean() - results.std())) / (results.mean() - results.std())*100)
  print('________________')

AGXY3.SA : 5.09
Cotação 1º dia de IPO:  10.11
Mínima histórica:  5.09 . Máxima histórica: 11.73
Variação Máxima % desde o IPO:  130.45
Média da cotação histórica indica o seguinte preço:  8.36
Mediana da cotação histórica indica o seguinte preço:  8.35
Mediana do Bootstrap indica o seguinte preço:  8.35
Distribuição do Bootstrap comparado à cotação histórica estima um viés de:  -0.02 no preço atual
Desvio Padrão do Bootstrap:  0.12 . Isso indica um preço histórico entre: 8.21  e: 8.46
Isso indica uma variação percentual média, desde o IPO de:  2.973641675234857
________________
ALLD3.SA : 6.9
Cotação 1º dia de IPO:  15.03
Mínima histórica:  4.5 . Máxima histórica: 33.47
Variação Máxima % desde o IPO:  385.07
Média da cotação histórica indica o seguinte preço:  12.05
Mediana da cotação histórica indica o seguinte preço:  11.14
Mediana do Bootstrap indica o seguinte preço:  11.14
Distribuição do Bootstrap comparado à cotação histórica estima um viés de:  -0.1 no preço atual
Desvio Padrão

#

In [1]:
# Das 45 empresas, somente as 11 abaixo possuem cotação atual maior que a cotação no dia do IPO.

cotatual_maior_ipo = {'Ticker':  ['CXSE3', 'GGPS3', 'INTB3', 'JALL3', 'NUBR33', 'ORVR3', 'RECV3', 'TTEN3', 'VAMO3', 'VITT3', 'VVEO3'],
      'Atividade' : ['Seguros', 'Administração Patrimonial', 'Segurança da informação e Energia', 'Tecnologia Sucroenergética', 'Banco (fintech)', 'Bioenergia, Energia Renovável e Materiais Recicláveis', 'Petróleo em terra e Gás natural', 'Agronegócio', 'Locação/Venda de caminhões, máquinas e equipamentos', 'Agronegócio', 'Holding (Grupo Mafra), Distribuidora de laboratórios e Logística hospitalar'],}

len(cotatual_maior_ipo)

2

In [8]:
pd.DataFrame(cotatual_maior_ipo)

,Ticker,Atividade
0,CXSE3,Seguros
1,GGPS3,Administração Patrimonial
2,INTB3,Segurança da informação e Energia
3,JALL3,Tecnologia Sucroenergética
4,NUBR33,Banco (fintech)
5,ORVR3,"Bioenergia, Energia Renovável e Materiais Reci..."
6,RECV3,Petróleo em terra e Gás natural
7,TTEN3,Agronegócio
8,VAMO3,"Locação/Venda de caminhões, máquinas e equipam..."
9,VITT3,Agronegócio


# 4) Conclusão

1) Todas as 45 empresas apresentaram alta variação percentual entre a máxima histórica e o preço atual. Tal fato é corroborado por um preço de IPO esticado e a necessidade do desenvolvimento de uma estratégia de algo trading operando short ( ou seja, "apostando" na queda do preço pós IPO e principalmente na queda drástica do preço pós máxima. Ok, sabemos que não é fácil.).


2) Somente 11 empresas apresentaram cotação atual maior que a cotação no 1º dia do IPO.

Tal fato indica, em princípio, percepção de valor por parte do investidor.
Isso se reflete no preço atual das empresas conforme o dataframe 'cotatual_maior_ipo'. Tais atividades são o agronegócio e suas tecnologias.

